In [9]:
import pandas as pd
import os
import git


def get_git_root():
    try:
        # Create a GitPython Repo object
        repo = git.Repo(search_parent_directories=True)

        # Get the absolute path of the root folder
        git_root = repo.git.rev_parse("--show-toplevel")

        return git_root
    except git.InvalidGitRepositoryError:
        # Handle the case where the current directory is not a Git repository
        print("Not a Git repository or an error occurred.")
        return None

# Get the root folder of the Git repository
root_folder = get_git_root()
# load CSV file from dataset dir

dataset_processed_dir=os.path.join(root_folder,"data/processed")
dataset_transform_dir=os.path.join(root_folder,"data/transform")
zone_processed_file='zone.csv'
dataset_processed_file="processed.csv"
dataset_processed_file_path=os.path.join(dataset_processed_dir,dataset_processed_file)
zone_processed_file_path=os.path.join(dataset_processed_dir,zone_processed_file)
zone=pd.read_csv(zone_processed_file_path)
print(zone.head())
df = pd.read_csv(dataset_processed_file_path)
df.head()


   LocationID        Borough                     Zone service_zone
0           1            EWR           Newark Airport          EWR
1           2         Queens              Jamaica Bay    Boro Zone
2           3          Bronx  Allerton/Pelham Gardens    Boro Zone
3           4      Manhattan            Alphabet City  Yellow Zone
4           5  Staten Island            Arden Heights    Boro Zone


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1,0.97,1,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1,1.10,1,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1,2.51,1,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0,1.90,1,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1,1.43,1,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [12]:
#data transform and data loading
# Dimension Tables:

#         Time Dimension:
#             time_id (primary key)
#             pickup_datetime (from tpep_pickup_datetime)
#             dropoff_datetime (from tpep_dropoff_datetime)

        
#         Location Dimension:

#             location_id (primary key)
#             location_name (derived from the "Zone" column in the reference table)
#             borough (derived from the "Borough" column in the reference table)
#             service_zone (derived from the "service_zone" column in the reference table)

#         Payment Type Dimension:
#             payment_type_id (primary key)
#             payment_type_name (from payment_type)

#         Rate Code Dimension:
#             rate_code_id (primary key)
#             rate_code_description (from RatecodeID)

#         Vendor Dimension:
#             vendor_id (primary key)
#             vendor_name (from VendorID)

#     Fact Table:

#         Trip Fact:
#             trip_id (primary key)
#             time_id (foreign key referencing Time Dimension)
#             location_pickup_id (foreign key referencing Location Dimension for pickup)
#             location_dropoff_id (foreign key referencing Location Dimension for dropoff)
#             passenger_count
#             trip_distance
#             rate_code_id (foreign key referencing Rate Code Dimension)
#             store_and_fwd_flag
#             payment_type_id (foreign key referencing Payment Type Dimension)
#             fare_amount
#             extra
#             mta_tax
#             tip_amount
#             tolls_amount
#             improvement_surcharge
#             total_amount
#             congestion_surcharge
#             airport_fee

#Time Dimension
time_dimention=df[['tpep_pickup_datetime','tpep_dropoff_datetime']].reset_index(drop=True)
time_dimention['time_id'] = time_dimention.index
time_dimention.head()
dataset_transform_file_path=os.path.join(dataset_transform_dir,"time_dem.csv")
time_dimention.to_csv(dataset_transform_file_path, index=False)

In [11]:
#Location Dimension
location_dimension=zone[['LocationID','Borough','Zone','service_zone']]
location_dimension.rename(columns={'LocationID':'location_id','Borough':'borough','Zone':'location_name'},inplace=True)
print(location_dimension.head())
dataset_transform_file_path=os.path.join(dataset_transform_dir,"locaton_dem.csv")
location_dimension.to_csv(dataset_transform_file_path, index=False)

   location_id        borough            location_name service_zone
0            1            EWR           Newark Airport          EWR
1            2         Queens              Jamaica Bay    Boro Zone
2            3          Bronx  Allerton/Pelham Gardens    Boro Zone
3            4      Manhattan            Alphabet City  Yellow Zone
4            5  Staten Island            Arden Heights    Boro Zone


In [13]:
#Vendor Dimension
vendor={
    1 : "Creative Mobile Technologies, LLC",
    2 : "VeriFone Inc."
}
#it specify we are making copy of slice of dataframe from original dataframe
vendor_dimension=df[['VendorID']].copy()

vendor_dimension.drop_duplicates(inplace=True)
vendor_dimension=vendor_dimension.reset_index(drop=True)
vendor_dimension.rename(columns={"VendorID":"vendor_id"},inplace=True)
vendor_dimension['vendor_name']=vendor_dimension['vendor_id'].map(vendor)
vendor_dimension=vendor_dimension[["vendor_id","vendor_name"]]
print(vendor_dimension)
dataset_transform_file_path=os.path.join(dataset_transform_dir,"vendor_dem.csv")
vendor_dimension.to_csv(dataset_transform_file_path, index=False)

   vendor_id                        vendor_name
0          2                      VeriFone Inc.
1          1  Creative Mobile Technologies, LLC


In [14]:
#after analysing data 99 seems to be personal ride
rate_code_type = {
    1:"Standard rate",
    2:"JFK",
    3:"Newark",
    4:"Nassau or Westchester",
    5:"Negotiated fare",
    6:"Group ride",
    99:"personal"
}

rate_code_dim = df[['RatecodeID']].copy().drop_duplicates().reset_index(drop=True)
rate_code_dim.rename(columns={"RatecodeID":"rate_code_id"},inplace=True)
rate_code_dim['rate_code_name'] = rate_code_dim['rate_code_id'].map(rate_code_type)
rate_code_dim = rate_code_dim[['rate_code_id','rate_code_name']]
print(rate_code_dim)
dataset_transform_file_path=os.path.join(dataset_transform_dir,"rate_code_dem.csv")
rate_code_dim.to_csv(dataset_transform_file_path, index=False)

   rate_code_id         rate_code_name
0             1          Standard rate
1             2                    JFK
2            99               personal
3             5        Negotiated fare
4             4  Nassau or Westchester
5             3                 Newark
6             6             Group ride


In [15]:
#Payment Type Dimension:
#             payment_type_id (primary key)
#             payment_type_name (from payment_type)
payment_type_name = {
    1:"Credit card",
    2:"Cash",
    3:"No charge",
    4:"Dispute",
    5:"Unknown",
    6:"Voided trip"
}
payment_type_dim = df[['payment_type']].copy()
payment_type_dim.drop_duplicates(inplace=True)
payment_type_dim=payment_type_dim.reset_index(drop=True)
payment_type_dim.rename(columns={"payment_type":"payment_type_id"},inplace=True)
payment_type_dim['payment_type_name'] = payment_type_dim['payment_type_id'].map(payment_type_name)
payment_type_dim = payment_type_dim[['payment_type_id','payment_type_name']]
print(payment_type_dim)
dataset_transform_file_path=os.path.join(dataset_transform_dir,"payment_type_dem.csv")
payment_type_dim.to_csv(dataset_transform_file_path, index=False)

   payment_type_id payment_type_name
0                2              Cash
1                1       Credit card
2                4           Dispute
3                3         No charge


In [ ]:
#fact table